In [ ]:
import torch
import os
import cv2
import grayscale_moments
import resnet
import hog

In [ ]:
data1_path = '../data/Part1/Part1'
data2_path = '../data/Part2/Part2'

model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
model.eval()

In [ ]:
# Create a dictionary to store the results
extracted_features = {
    'file_path': [],
    'class': [],
    'cm': [],
    'hog': [],
    'avgpool': [],
    'layer3': [],
    'fc': [],
}

In [ ]:
extracted_features = []
# For each image in data1_path extract the features
for root, dirs, files in os.walk(data1_path):
    for image in files:
        if not image.endswith('.jpg'):
            continue

        # Class can be glioma, menin, tumor. Get the class from the filename
        class_name = image.split('_')[0]+'_' + image.split('_')[1]
        image_path = os.path.join(f'{class_name}/', image)
        # Get the full path to the image
        image_path = os.path.abspath(os.path.join(data1_path, image_path))
        print(f'Processing {image_path}')

        # Load the image
        img = cv2.imread(image_path, cv2.IMREAD_COLOR)
        grayscale_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        blocks = grayscale_moments.blockify(grayscale_img, new_H=300, new_W=100, block_H=30, block_W=10)

        # For each block, calculate the color moments, return a tensor of shape [(3*blocks)]
        color_moments = []
        for block in blocks:
            moments = grayscale_moments.moments(block.numpy())
            color_moments.extend(moments)  # Append the moments directly to the list
        color_moments = torch.tensor(color_moments)  # Convert the list to a tensor

        print(f'Color moments: {color_moments.shape}')

        hog_features = hog.hog(grayscale_img)

        preprocessed_img = resnet.preprocess_image(img)

        resnet_features = resnet.extract_layer_features(model, preprocessed_img, layer_names=['avgpool', 'layer3', 'fc'])
        avgpool = resnet_features['avgpool']
        layer3 = resnet_features['layer3']
        fc = resnet_features['fc']
        avgpool = avgpool.view(avgpool.size(1))
        avgpool = torch.mean(avgpool.view(-1, 2), dim=1)
        layer3 = torch.mean(layer3.view(-1, 1024, 14*14), dim=2)
        layer3 = layer3.view(layer3.size(1))
        fc = fc.view(fc.size(1))
        
        features_dict = {
            'file_path': image_path,
            'class': class_name,
            'cm': color_moments,
            'hog': hog_features,
            'avgpool': avgpool,
            'layer3': layer3,
            'fc': fc
        }
        # Append the features to the list
        extracted_features.append(features_dict)


In [ ]:
# Save the extracted features into a pt file
torch.save(extracted_features, '../data/extracted_features.pt')

In [ ]:
# Load the extracted features from the pt file
extracted_features = torch.load('../data/extracted_features.pt')
# Print the extracted features
print(extracted_features)